## Preparations

После перезагрузки компьютера, запустите следующие команды в терминале (пароль 1qaz!QAZ).
```bash
echo 1024 | sudo tee /sys/module/usbcore/parameters/usbfs_memory_mb
sudo cat /sys/module/usbcore/parameters/usbfs_memory_mb
```

В трех отдельных сессиях терминала запустите, если еще не запущено, по одной строке из
```bash
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh all
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh info
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh err
```
В этих сессиях будут отображаться (1) все сообщения с информацией о сканировании, (2) сообщения об основных этапах сканирования, (3) ошибки.

## Initialize cameras

1. Запустите код в следующей клетке, проверьте по логам что все 6 камер снимают изображения.

In [ ]:
from pathlib import Path
import datetime
import sys

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.scanning.scan_helper.scan_helper import ScanHelper


scans_root = '/mnt/data/sk3d/raw_scans'
today = datetime.datetime.now().strftime('%y_%m_%d')
logfile = f'/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/logs/scanning/{today}_cameras.log'
started_warmup_at = datetime.datetime.now()

scan_helper = ScanHelper(scans_root, logfile, trajectories='human_sphere', thread='cameras')

## Set the white balance of the cameras

Настройте баланс белого один раз после запуска тетрадки.
1. Выключите свет в комнате.
2. Запустите код в следующей клетке и следуйте инструкциям.

In [ ]:
robot = scan_helper.camera_trajectory_class()
robot.move_for_awb(.2)
scan_helper.set_white_balance()
robot.rest(.2)
del robot

## Scan

1. Задайте имя текущей сцены из таблицы.
2. Проверьте правильность расположения ламп.
3. Загрузите настройки камер с диска.

In [ ]:
scene_name = 'people_debug'
scan_helper.load_camera_settings(scene_name)

4. Запустите съемку камерами. При перезапуске, сканирование продолжится с того положения на котором было прервано.

Если Вы хотите запустить сканирование сначала и переснять уже полученные изображения, то перед сканированием запустите
```python
scan_helper.reset_camera_scanning_status(scene_name)
```

In [ ]:
scan_helper.scan_cameras(scene_name)

5. Скачайте данные с телефонов, как описано в видеоинструкции.

## Stop scanning

1. Для остановки камер в конце смены или перед перезапуском тетрадки запустите код ниже.
Если клетка не завершается в течение 10 секунд, и камеры продолжают снимать, то, скорее всего, клетка зависла --- смотрите шаги 2-4.
Если клетка не зависла, переходите к шагу 5.

In [ ]:
scan_helper.stop()

Если на шаге 1 установка зависла

2. Перезапустите ядро тетрадки.
3. Запустите код ниже.
4. Если клетка не завершается в течение 10 секунд, повторите с шага 2.

In [ ]:
import datetime
import sys

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.scanning.scan_helper.scan_helper import ScanHelper

scans_root = '/mnt/data/sk3d/raw_scans'
today = datetime.datetime.now().strftime('%y_%m_%d')
logfile = f'/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/logs/scanning/{today}_cameras.log'
scan_helper = ScanHelper(scans_root, logfile, trajectories='tabletop', thread='cameras')
scan_helper.stop()

5. Перезапустите ядро тетрадки.
6. Закройте окно с графиками параметров телефонов.
7. Если робот не находится в стартовом положении, переведите его в стартовое положение как показано в видеоинструкции.
8. Выключите робота через пульт.
9. Выключите сетевой фильтр который питает лампы, и выдерните его из розетки.
10. Проверьте что экраны телефонов погасли. Если нет --- выключите телефоны кнопкой.